In [1]:
# Stage 1: Collect Explore
# Stage 2: Annotate and convert labelme annotations to seg
# Stage 3: Find spawn location and target for reexplore (reexplore_data.json)
# Stage 4: Run reexplore for all objects, one at a time 
# Stage 5: Run label prop
# Stage 6: Run training 

In [2]:
# Stage 1: Collect Explore to $HOME/explore_data

# Stage 3: Find spawn location and target for reexplore (reexplore_data.json) in a separate out_dir


In [3]:
# Stage 5: Run label prop
# Ensure reex_dir has all objects reexplored

# reex_dir = '/home/locobotm/explore_data/default/0/reexplore/'
# explore_dir = '/home/locobotm/explore_data/default/0/'

# soumiths data
reex_dir = '/home/locobotm/0512_data/reexplore_data_0512_3/'
explore_dir = '/home/locobotm/home2_data/0'

# load reex json
import os
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np
from droidlet.perception.robot import LabelPropagate
from PIL import Image

d3_40_colors_rgb: np.ndarray = np.array(
    [
        [31, 119, 180],
        [174, 199, 232],
        [255, 127, 14],
        [255, 187, 120],
        [44, 160, 44],
        [152, 223, 138],
        [214, 39, 40],
        [255, 152, 150],
        [148, 103, 189],
        [197, 176, 213],
        [140, 86, 75],
        [196, 156, 148],
        [227, 119, 194],
        [247, 182, 210],
        [127, 127, 127],
        [199, 199, 199],
        [188, 189, 34],
        [219, 219, 141],
        [23, 190, 207],
        [158, 218, 229],
        [57, 59, 121],
        [82, 84, 163],
        [107, 110, 207],
        [156, 158, 222],
        [99, 121, 57],
        [140, 162, 82],
        [181, 207, 107],
        [206, 219, 156],
        [140, 109, 49],
        [189, 158, 57],
        [231, 186, 82],
        [231, 203, 148],
        [132, 60, 57],
        [173, 73, 74],
        [214, 97, 107],
        [231, 150, 156],
        [123, 65, 115],
        [165, 81, 148],
        [206, 109, 189],
        [222, 158, 214],
    ],
    dtype=np.uint8,
) 

def save_propagated_visual(semantic1, semantic2, save_dir, out_indx):
    
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    arr = []
    # arr.append(semantic1[0].reshape(480, 640))
    # arr.append(
    for semantic_obs in [semantic1.T[0].reshape(480,640), semantic2]:
        print(f'type(semantic_obs) {semantic_obs.shape}')
        
        
        semantic_img = Image.new("P", (semantic_obs.shape[0], semantic_obs.shape[1]))
        semantic_img.putpalette(d3_40_colors_rgb.flatten())
        semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
        semantic_img = semantic_img.convert("RGBA")
        arr.append(semantic_img)

    titles = ['uncompute_pcd', 'propagated']
    plt.figure(figsize=(12 ,8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 2, i+1)
        ax.axis('off')
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.savefig("{}/{:05d}.jpg".format(save_dir, out_indx))
    plt.show()


reex_json = os.path.join(reex_dir, 'reexplore_data.json')
with open(reex_json, 'r') as f:
    reex_data = json.load(f)

lp = LabelPropagate()
    
for reex_id in ['4']: #reex_data.keys():
    obj_dir = os.path.join(reex_dir, reex_id)
    obj_data = reex_data[reex_id]
    print(f'obj_dir {obj_dir}')
    src_img_indx = obj_data['src_img_id']
    # for each examine heuristic
    
    # load src_img_id rgb, label and pose
    src_img = cv2.imread(os.path.join(explore_dir, "rgb/{:05d}.jpg".format(src_img_indx)))
    src_depth = np.load(os.path.join(explore_dir, "depth/{:05d}.npy".format(src_img_indx)))
    src_pcd = np.load(os.path.join(explore_dir, "pcd/{:05d}.npy".format(src_img_indx)))
    src_label = np.load(os.path.join(explore_dir, "seg/{:05d}.npy".format(src_img_indx)))
    
    src_label = np.rot90(src_label, k=-1, axes=(1, 0))
    src_depth = np.rot90(src_depth, k=-1, axes=(1, 0))
    src_img = np.rot90(src_img, k=-1, axes=(1, 0))
    
    with open(os.path.join(explore_dir, 'data.json'), "r") as f:
        base_pose_data = json.load(f)
    src_pose = base_pose_data["{}".format(src_img_indx)]
    
    # for each heuristic, for each image in rgb, propagate 
    for heu in ['c1pp']:
        heu_dir = os.path.join(obj_dir, heu)
        prop_dir = os.path.join(heu_dir, 'pred')
        print(f'saving propagated frames to {prop_dir}')
        os.makedirs(prop_dir, exist_ok=True)

        num_samples = min(len(os.listdir(os.path.join(heu_dir, 'rgb'))), 18)
        with open(os.path.join(heu_dir, 'data.json'), "r") as f:
            cur_pose_data = json.load(f) 

        for p in range(num_samples):
            cur_pose = cur_pose_data[str(p)]
            cur_depth = np.load(os.path.join(heu_dir, "depth/{:05d}.npy".format(p)))    
#             cur_depth = np.load(os.path.join(heu_dir, "depth/{:05d}.npy".format(p)))
            cur_img = cv2.imread(os.path.join(heu_dir, "rgb/{:05d}.jpg".format(p)))
            
            cur_depth = np.rot90(cur_depth, k=-1, axes=(1, 0))
            cur_img = np.rot90(cur_img, k=-1, axes=(1, 0))
            
            annot_img, upcd = lp(src_img, src_depth, src_label, src_pose, cur_pose, cur_depth)
            annot_img = np.rot90(annot_img, k=1, axes=(1, 0))
            
            print(annot_img.shape, annot_img.dtype)
            mask = np.zeros(annot_img.shape[:2], dtype="uint8")
            mask[annot_img != 0] = True
            
            # cur_img = cv2.bitwise_or(cur_img, cur_img, mask=mask)

            np.save(os.path.join(prop_dir, "{:05d}.npy".format(p)), annot_img.astype(np.uint32))
            save_propagated_visual(upcd, annot_img, os.path.join(prop_dir, 'lp_visuals'), p)
            
         #FIXME: also save the gt label

        
# TODO: combine all label prop for all objects in range(0,18,2) pred folders
# TODO: run coco and training on each pred folder
    

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2022-05-13 18:55:00,277 - default_behaviors - Seed chosen: 116819841


obj_dir /home/locobotm/0512_data/reexplore_data_0512_3/4
saving propagated frames to /home/locobotm/0512_data/reexplore_data_0512_3/4/c1pp/pred
height 480 width 640
uv_one_in_cam.shape (3, 307200)
rgbd.depth.shape (480, 640), pts.shape (480, 640, 3)
pts_in_cam.shape from uncompute_pcd (3, 307200)


NameError: name 'cur_pts_in_cur_cam' is not defined

In [ ]:
921600/3

In [ ]:
640*480